## Working with an LLM programmatically

You have certainly interacted before with a Large Language Model (LLM) like ChatGPT. This is usually done through a UI or an application.

In this Notebook, we are going to use Python to connect and query an LLM directly through its API. For this Lab we have selected the model **Mistral-7B Instruct v2**.(https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2). This is a fully Open Source model (Apache 2.0 license), that although much lighter than other commercial or open source models is very capable, especially with the tasks we intend to use it for.

This model has already been deployed on the Lab cluster because even if it's a smaller model, it still needs a GPU with 24GB of RAM to run...

In [1]:
!pip install -q langchain==0.1.14


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages. We will then import the libraries we need.

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 96 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too "creative". After all, we're not trying to write a fancy poem here!

In [3]:
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:11434"

In [4]:
# LLM definition
llm = Ollama(
    base_url=inference_server_url,
    model="mistral",
    top_p=0.92,
    temperature=0.01,
    num_predict=512,
    repeat_penalty=1.03,
    callbacks=[StreamingStdOutCallbackHandler()]
)

print(llm)

Ollama
Params: {'model': 'mistral', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': 512, 'repeat_last_n': None, 'repeat_penalty': 1.03, 'temperature': 0.01, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': 0.92}, 'system': None, 'template': None, 'keep_alive': None}


We also need a **template** to be applied to every request we are sending to the model (the "Prompt").

When querying a model, you almost never want to send directly what the user has typed. On top of this entry, you need to give proper instructions to the model so that it knows how to handle it: what and how to answer, what NOT to answer, the tone it must use...

In [5]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchain allows us to now easily "stitch" those elements together and create a **conversation** object that we will use to query the model.

In [6]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the model!

In [ ]:
query = "What is Artificial Intelligence?"
conversation.predict(input=query); # ";" at the end of the line hides final output (repetion of the streamed answer)

We will come back to this notebook at section 3.5 for some exercises, so you can leave it open for the moment.

In [8]:
# Read the claims and populate a dictionary
claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # Opening JSON file
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data


In [9]:
for filename in onlyfiles:
    print(f"***************************")
    print(f"* Claim: {filename}")
    print(f"***************************")
    print("Original content:")
    print("-----------------")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('Summary:')
    print("--------")
    summary_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    conversation.predict(input=summary_input);
    print("\n\n                          ----====----\n")

***************************
* Claim: claim3.json
***************************
Original content:
-----------------
Subject: Urgent: Car Accident Claim Assistance Needed
Content:
Dear AssuranceCare Inc.,\n\nI hope this email finds you well. I'm writing to, uh, inform you about, well, something that happened recently. It's, um, about a car accident, and I'm not really sure how to, you know, go about all this. I'm kinda anxious and confused, to be honest.\n\nSo, the accident, uh, occurred on January 15, 2024, at around 3:30 PM. I was driving, or, um, attempting to drive, my car at the intersection of Maple Street and Elm Avenue. It's kinda close to, um, a gas station and a, uh, coffee shop called Brew Haven? I'm not sure if that matters, but, yeah.\n\nSo, I was just, you know, driving along, and suddenly, out of nowhere, another car, a Blue Crest Sedan, crashed into the, uh, driver's side of my car. It was like, whoa, what just happened, you know? There was this screeching noise and, uh, so

In [12]:
# Analyze the claims

for filename in onlyfiles:
    print(f"***************************")
    print(f"* Claim: {filename}")
    print(f"***************************")
    print("Original content:")
    print("-----------------")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('Analysis:')
    print("--------")
    text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    sentiment_query = "What is the sentiment of the person sending this claim?"
    location_query = "Where does the event the claim is related to happen?"
    time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
    # Construct input dictionary
    input_data_sentiment = {
        'input': text_input,  # Text input
        'query': sentiment_query  # Query
    }    
    input_data_location = {
        'input': text_input,  # Text input
        'query': location_query  # Query
    } 
    input_data_time = {
        'input': text_input,  # Text input
        'query': time_query  # Query
    }     
    print(f"- Sentiment: ")
    conversation.predict(input=input_data_sentiment);
    print("\n- Location: ")
    conversation.predict(input=input_data_location);
    print("\n- Time: ")
    conversation.predict(input=input_data_time);
    print("\n\n                          ----====----\n")

***************************
* Claim: claim3.json
***************************
Original content:
-----------------
Subject: Urgent: Car Accident Claim Assistance Needed
Content:
Dear AssuranceCare Inc.,\n\nI hope this email finds you well. I'm writing to, uh, inform you about, well, something that happened recently. It's, um, about a car accident, and I'm not really sure how to, you know, go about all this. I'm kinda anxious and confused, to be honest.\n\nSo, the accident, uh, occurred on January 15, 2024, at around 3:30 PM. I was driving, or, um, attempting to drive, my car at the intersection of Maple Street and Elm Avenue. It's kinda close to, um, a gas station and a, uh, coffee shop called Brew Haven? I'm not sure if that matters, but, yeah.\n\nSo, I was just, you know, driving along, and suddenly, out of nowhere, another car, a Blue Crest Sedan, crashed into the, uh, driver's side of my car. It was like, whoa, what just happened, you know? There was this screeching noise and, uh, so